In [ ]:
! pip install pyspark

In [3]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, concat, lit

In [4]:
spark = SparkSession.builder.getOrCreate()

In [ ]:
# conexão com o drive realizada pela interface gráfica

In [5]:
produtos = spark.read.csv('drive/MyDrive/Colab Notebooks/colab_ebac/spark_data/produtos.csv', inferSchema=True, header=True)
clientes = spark.read.csv('drive/MyDrive/Colab Notebooks/colab_ebac/spark_data/clientes.csv', inferSchema=True, header=True)
itens_pedido = spark.read.csv('drive/MyDrive/Colab Notebooks/colab_ebac/spark_data/itens_pedido.csv', inferSchema=True, header=True)
pedidos = spark.read.csv('drive/MyDrive/Colab Notebooks/colab_ebac/spark_data/pedidos.csv', inferSchema=True, header=True)

In [11]:
join_pedidos_clientes_df = pedidos.join(clientes, pedidos.id_cliente == clientes.id_cliente, 'inner')  # Se o nome da coluna for diferente entre as tabelas
join_pedidos_clientes_df = pedidos.join(clientes, 'id_cliente')  # Se o nome da coluna for igual nas duas tabelas

clientes.show(5)
pedidos.show(5)
join_pedidos_clientes_df.show(5)

+--------------------+--------------------+-----------+--------------------+--------------+
|          id_cliente|    id_unico_cliente|cep_cliente|      cidade_cliente|estado_cliente|
+--------------------+--------------------+-----------+--------------------+--------------+
|06b8999e2fba1a1fb...|861eff4711a542e4b...|      14409|              franca|            SP|
|18955e83d337fd6b2...|290c77bc529b7ac93...|       9790|sao bernardo do c...|            SP|
|4e7b3e00288586ebd...|060e732b5b29e8181...|       1151|           sao paulo|            SP|
|b2b6027bc5c5109e5...|259dac757896d24d7...|       8775|     mogi das cruzes|            SP|
|4f2d8ab171c80ec83...|345ecd01c38d18a90...|      13056|            campinas|            SP|
+--------------------+--------------------+-----------+--------------------+--------------+
only showing top 5 rows

+--------------------+--------------------+-------------+-------------------+---------------------+-------------------------+--------------------+-

In [12]:
# Join com três tabelas
join_itens_pedido_df = itens_pedido.join(pedidos, 'id_pedido').join(produtos, 'id_produto')
join_itens_pedido_df.show(5)

+--------------------+--------------------+--------------+--------------------+-------------------+-----+-----------+--------------------+-------------+-------------------+---------------------+-------------------------+--------------------+---------------------+------------------+--------------------+-------------------------+------------------------+--------------+----------------------+-----------------+------------------+
|          id_produto|           id_pedido|item_id_pedido|         id_vendedor|  data_limite_envio|preco|valor_frete|          id_cliente|status_pedido| data_compra_pedido|data_aprovacao_pedido|data_envio_transportadora|data_entrega_cliente|data_estimada_entrega| categoria_produto|tamanho_nome_produto|tamanho_descricao_produto|quantidade_fotos_produto|peso_produto_g|comprimento_produto_cm|altura_produto_cm|largura_produto_cm|
+--------------------+--------------------+--------------+--------------------+-------------------+-----+-----------+--------------------+--

In [14]:
# retonar uma tabela com todos os pedidos cancelados e clientes
# poderíamos utilizar o SELECT mas também podemos utilizar o .filter
join_pedidos_cancelados_df = pedidos.filter(col('status_pedido') == 'canceled').join(clientes, 'id_cliente')
join_pedidos_cancelados_df.show(5)

+--------------------+--------------------+-------------+-------------------+---------------------+-------------------------+--------------------+---------------------+--------------------+-----------+--------------+--------------+
|          id_cliente|           id_pedido|status_pedido| data_compra_pedido|data_aprovacao_pedido|data_envio_transportadora|data_entrega_cliente|data_estimada_entrega|    id_unico_cliente|cep_cliente|cidade_cliente|estado_cliente|
+--------------------+--------------------+-------------+-------------------+---------------------+-------------------------+--------------------+---------------------+--------------------+-----------+--------------+--------------+
|6d6b50b66d79f8082...|1b9ecfe83cdc25925...|     canceled|2018-08-04 14:29:27|  2018-08-07 04:10:26|                     NULL|                NULL|  2018-08-14 00:00:00|8ea097b1824dbd4d1...|       5514|     sao paulo|            SP|
|e3fe72696c4713d64...|714fb133a6730ab81...|     canceled|2018-01-26 21:3

In [17]:
# Criando um novo identificador com o concat
pedidos_concat_df = pedidos.withColumn('id_pedido_cliente', concat(col('id_pedido'), lit('-'), col('id_cliente')))

# A nova coluna será adicionada na última posição, então vamos trazê-la para a primeira posição
print(pedidos_concat_df.columns)

# Criando a lista das colunas na ordem desejada
colunas = pedidos_concat_df.columns
colunas.remove('id_pedido_cliente')
colunas.insert(0, 'id_pedido_cliente')
print(colunas)

# Reorganizando a ordem das colunas do DataFrame
pedidos_reorganizar_df = pedidos_concat_df.select(colunas)
pedidos_reorganizar_df.show(5)

['id_pedido', 'id_cliente', 'status_pedido', 'data_compra_pedido', 'data_aprovacao_pedido', 'data_envio_transportadora', 'data_entrega_cliente', 'data_estimada_entrega', 'id_pedido_cliente']
['id_pedido_cliente', 'id_pedido', 'id_cliente', 'status_pedido', 'data_compra_pedido', 'data_aprovacao_pedido', 'data_envio_transportadora', 'data_entrega_cliente', 'data_estimada_entrega']
+--------------------+--------------------+--------------------+-------------+-------------------+---------------------+-------------------------+--------------------+---------------------+
|   id_pedido_cliente|           id_pedido|          id_cliente|status_pedido| data_compra_pedido|data_aprovacao_pedido|data_envio_transportadora|data_entrega_cliente|data_estimada_entrega|
+--------------------+--------------------+--------------------+-------------+-------------------+---------------------+-------------------------+--------------------+---------------------+
|e481f51cbdc54678b...|e481f51cbdc54678b...|9ef43

In [18]:
join_itens_pedido_df.write.mode('overwrite').option('header', 'true').parquet('/drive/MyDrive/Colab Notebooks/colab_ebac/spark_data/output/join_itens_pedido_parquet')

In [19]:
spark.read.option('header', 'true').parquet('/drive/MyDrive/Colab Notebooks/colab_ebac/spark_data/output/join_itens_pedido_parquet').show(5)

+--------------------+--------------------+--------------+--------------------+-------------------+-----+-----------+--------------------+-------------+-------------------+---------------------+-------------------------+--------------------+---------------------+--------------------+--------------------+-------------------------+------------------------+--------------+----------------------+-----------------+------------------+
|          id_produto|           id_pedido|item_id_pedido|         id_vendedor|  data_limite_envio|preco|valor_frete|          id_cliente|status_pedido| data_compra_pedido|data_aprovacao_pedido|data_envio_transportadora|data_entrega_cliente|data_estimada_entrega|   categoria_produto|tamanho_nome_produto|tamanho_descricao_produto|quantidade_fotos_produto|peso_produto_g|comprimento_produto_cm|altura_produto_cm|largura_produto_cm|
+--------------------+--------------------+--------------+--------------------+-------------------+-----+-----------+-------------------

In [20]:
spark.stop()